In [ ]:
# This cell is NOT editable. Overwrite variables on your own discretion.
# Any changes other than the script code will NOT BE SAVED!
# All cells are assumed to be script code cells, unless explictly tagged as 'o9_ignore'

In [2]:
_input_df = "select (Version.[Version Name]*Product.[Product].Filter(#.Name in {[196426],[208821]})*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});"
_predict_df = "select (Version.[Version Name]*Product.[Product].Filter(#.Name in {[196426],[208821]})*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});"


# Initialize the O9DataLake with the input parameters and dataframes
# Data can be accessed with O9DataLake.get(<Input Name>)
# Overwritten values will not be reflected in the O9DataLake after initialization

from o9_common_utils.O9DataLake import O9DataLake, ResourceType, DataSource,PluginSetting
input_df = O9DataLake.register("input_df",data_source = DataSource.LS, entity_type = ResourceType.IBPL, query = _input_df,plugin_setting = PluginSetting.Inputs)
predict_df = O9DataLake.register("predict_df",data_source = DataSource.LS, entity_type = ResourceType.IBPL, query = _predict_df,plugin_setting = PluginSetting.Inputs)

O9DataLake.register("out_df",data_source = DataSource.LS,entity_type = ResourceType.IBPL,plugin_setting = PluginSetting.Outputs)
script_params = O9DataLake.register({}, data_source = DataSource.LS,plugin_setting = PluginSetting.ScriptParam)

In [19]:
import logging
import os
from sklearn import tree
logger = logging.getLogger('o9_logger')
logger.info('In User script from jhub')
out_df = None
import logging
import sys
import time

logger = logging.getLogger('o9_logger')
logger.info("sys path")
logger.info(sys.path)
import logging
logger = logging.getLogger("o9_logger")

logger.info("Hello: We are in main script")

logger.error("THIS IS TEST ERROR MESSAGE")
logger.info("THIS IS TEST INFO MESSAGE")
logger.warn("THIS IS TEST WARNING MESSAGE")
logger.debug("THIS IS TEST DEBUG MESSAGE")

#logger.error(f'env var {os.environ}')

#time.sleep(50)

if predict_df is not None and len(predict_df.index) > 0:
    x = input_df[['Mean Pricing Save PCT', 'Placement Count', 'Promotion Count', 'DPSellOutPrice']]
    y = input_df[['DPSellOutUnitsActuals']]
    x = x.fillna(0).values
    y = y.fillna(0).values

    clf = tree.DecisionTreeRegressor()
    clf = clf.fit(x, y)

    predict = predict_df[['Mean Pricing Save PCT', 'Placement Count', 'Promotion Count', 'DPSellOutPrice']].fillna(0).values
    predict_values = clf.predict(predict)
    predict_df["DPSellOutUnitsFcst"] = predict_values
    out_df=predict_df[['Time.[FiscalWeek]', 'Product.[Product]', 'Version.[Version Name]', 'SalesAccount.[Account]','Location.[Location]','DPSellOutUnitsFcst']]
    logger.info(out_df)
else:
    out_df = None

logger.info("Ingesting output")
logger.info(out_df.dtypes)
logger.info("Plugin ends")


2024-12-04 11:12:37,845 - o9_logger - INFO  - In User script from jhub
2024-12-04 11:12:37,846 - o9_logger - INFO  - sys path
2024-12-04 11:12:37,846 - o9_logger - INFO  - ['/root/o9plugins', '/opt/conda/envs/k8qa_sparkk8org1tenant1/lib/python310.zip', '/opt/conda/envs/k8qa_sparkk8org1tenant1/lib/python3.10', '/opt/conda/envs/k8qa_sparkk8org1tenant1/lib/python3.10/lib-dynload', '', '/opt/conda/envs/k8qa_sparkk8org1tenant1/lib/python3.10/site-packages', '/opt/conda/envs/k8qa_sparkk8org1tenant1/lib/python3.10/site-packages/sentencepiece-0.1.96-py3.10-linux-x86_64.egg']
2024-12-04 11:12:37,847 - o9_logger - INFO  - Hello: We are in main script
2024-12-04 11:12:37,847 - o9_logger - ERROR  - THIS IS TEST ERROR MESSAGE
2024-12-04 11:12:37,848 - o9_logger - INFO  - THIS IS TEST INFO MESSAGE
2024-12-04 11:12:37,848 - o9_logger - WARNING  - THIS IS TEST WARNING MESSAGE
2024-12-04 11:12:37,849 - o9_logger - DEBUG  - THIS IS TEST DEBUG MESSAGE
2024-12-04 11:12:37,874 - o9_logger - INFO  -      Ti

/tmp/ipykernel_3025/2769376959.py:21: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn("THIS IS TEST WARNING MESSAGE")
